In [1]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
# Load MNIST
mnist = input_data.read_data_sets("data/", one_hot=True)
trainimgs, trainlabels, testimgs, testlabels \
 = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels 
ntrain, ntest, dim, nclasses \
 = trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1], trainlabels.shape[1]
print ("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
istate = tf.placeholder("float", [None, 2*n_hidden])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input,n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden':tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
def RNN(x,istate,weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    
    H = tf.matmul(x, weights['hidden']) + biases['hidden']
    
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, H)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x,initial_state=istate)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x,istate,weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [23]:
# Launch the graph
sess = tf.Session()
sess.run(init)
step = 1
    # Keep training until reach max iterations
while step * batch_size < training_iters:
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, n_steps, n_input))
    # Run optimization op (backprop)
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, istate: np.zeros((batch_size, 2*n_hidden))})
    if step % display_step == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, istate: np.zeros((batch_size, 2*n_hidden))})
        # Calculate batch loss
        avg_cost += sess.run(cost, feed_dict={x: batch_x, y: batch_y 
                                          , istate: np.zeros((batch_size, 2*n_hidden))})/total_batch
        print "Iter " + str(step*batch_size) + ",Average cost = " + \
              "{:.6f}".format(avg_cost) + ", Training Accuracy= " + \
              "{:.5f}".format(acc)
                 # Calculate accuracy for 128 mnist test images
        test_len = 128
        test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
        test_label = mnist.test.labels[:test_len]
        print "Testing Accuracy:", \
            sess.run(accuracy, feed_dict={x: test_data, y: test_label, istate: np.zeros((batch_size, 2*n_hidden))})
    step += 1


print "Optimization Finished!"

    

Iter 1280,Average cost = 0.004540, Training Accuracy= 0.36719
Testing Accuracy: 0.359375
Iter 2560,Average cost = 0.004206, Training Accuracy= 0.32031
Testing Accuracy: 0.4375
Iter 3840,Average cost = 0.002869, Training Accuracy= 0.55469
Testing Accuracy: 0.539062
Iter 5120,Average cost = 0.002463, Training Accuracy= 0.65625
Testing Accuracy: 0.609375
Iter 6400,Average cost = 0.002281, Training Accuracy= 0.73438
Testing Accuracy: 0.640625
Iter 7680,Average cost = 0.002118, Training Accuracy= 0.67188
Testing Accuracy: 0.671875
Iter 8960,Average cost = 0.002093, Training Accuracy= 0.66406
Testing Accuracy: 0.6875
Iter 10240,Average cost = 0.001782, Training Accuracy= 0.75000
Testing Accuracy: 0.75
Iter 11520,Average cost = 0.001367, Training Accuracy= 0.84375
Testing Accuracy: 0.789062
Iter 12800,Average cost = 0.001477, Training Accuracy= 0.77344
Testing Accuracy: 0.804688
Iter 14080,Average cost = 0.001023, Training Accuracy= 0.87500
Testing Accuracy: 0.78125
Iter 15360,Average cost = 

In [25]:
# How may sequences will we use?
nsteps2     = 28
# Test
testimgs = testimgs.reshape((ntest, n_steps, n_input))
testimgs_trucated = np.zeros(testimgs.shape)
testimgs_trucated[:, 28-nsteps2:] = testimgs[:, :nsteps2, :]

test_acc = sess.run(accuracy, feed_dict={x: testimgs_trucated, y: testlabels, istate: np.zeros((ntest, 2*n_hidden))})
print (" If we use %d seqs, test accuracy becomes %.3f" % (nsteps2, test_acc))

 If we use 28 seqs, test accuracy becomes 0.958
